initialize_NLP() was modified from dev version to allow for kwargs...

In [3]:
using NLOptControl
using Plots
using Polynomials
using FastGaussQuadrature
using Parameters

In [4]:
# initialize basic problem definition
ps, nlp = initialize_NLP(numStates=1,numControls=1,Ni=2,Nck=[3, 3]);

In [3]:
nlp

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: [4,4]
  lengthStateVector: 8
  numControls: 1
  numControlPoints: [3,3]
  lengthControlVector: 6
  lengthDecVector: 16
  stateStartIdx: 1
  stateStopIdx: 8
  controlStartIdx: 9
  controlStopIdx: 14
  timeStartIdx: 15
  timeStopIdx: 16
  stateVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [4]:
ps

NLOptControl.PS_data
  Nck: [3,3]
  Ni: 2
  τ: Array{Float64,1}[[-1.0,-0.289898,0.689898],[-1.0,-0.289898,0.689898]]
  ω: Array{Float64,1}[[0.222222,1.02497,0.752806],[0.222222,1.02497,0.752806]]
  t0: 0.0
  tf: 0.0
  stateMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0; 0.0],

[0.0; 0.0; 0.0; 0.0]]
  controlMatrix: Array{Float64,2}[
[0.0; 0.0; 0.0],

[0.0; 0.0; 0.0]]

In [5]:
# define problem
α₁ =  -0.3; α₂ = 3; α₃ = -8; α₄ =  7;
@unpack t0, tf = ps;
t0 = 0.0;
tf = 10.0;
@pack ps = t0,tf;  
t = Array(linspace(t0,tf,100));

In [9]:
function create_intervals(t0::Float64,tf::Float64,Ni::Int64,Nck::Array{Int64,1},τ::Array{Vector{Float64},1},ω::Array{Vector{Float64},1})
  di = (tf - t0)/Ni; # interval size
  # create mesh points
  tm = zeros(Float64,Ni+1); tm[1] = t0;
  for idx in 1:Ni
    tm[idx+1] = tm[idx] + di;
  end
    # go through each mesh interval creating time intervals; [t(i-1),t(i)] --> [-1,1]
    ts=[[scale_tau(τ[int],tm[int],tm[int+1]);di*int] for int in 1:Ni];
    ωₛ=[scale_w(ω[int],tm[int],tm[int+1]) for int in 1:Ni];
    return di, tm, ts, ωₛ
end

create_intervals (generic function with 1 method)

In [10]:
@unpack Nck, Ni, t0, tf = ps
taus_and_weights = [gaussradau(Nck[int]) for int in 1:Ni];
τ = [taus_and_weights[int][1] for int in 1:Ni];
ω = [taus_and_weights[int][2] for int in 1:Ni];
di, tm, ts, ωₛ=create_intervals(t0,tf,Ni,Nck,τ,ω);
print(di,"\n")
print(tm,"\n")
print(ts,"\n")
print(ωₛ,"\n")

5.0
[0.0,5.0,10.0]
Array{Float64,1}[[0.0,1.77526,4.22474,5.0],[5.0,6.77526,9.22474,10.0]]
Array{Float64,1}[[0.555556,2.56243,1.88202],[0.555556,2.56243,1.88202]]
